In [1]:
!pip install -U langchain-ollama langchain-openai

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/review_dataset

__MACOSX  w_review_train.csv  w_review_train.parquet


In [4]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [5]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [6]:
df.shape

(40000, 2)

In [7]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [8]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [9]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [10]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [11]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [12]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.21 s, sys: 22.8 ms, total: 4.24 s
Wall time: 3.12 s


In [13]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [14]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.47 s, sys: 56.9 ms, total: 4.53 s
Wall time: 3.04 s


In [15]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [16]:
coffee_reviews_df.shape

(4981, 2)

In [17]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [19]:
!ollama pull gemma3:270m

In [20]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
gemma3:270m                           e7d36fb2c3b3    291 MB    Less than a second ago    
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    8 minutes ago             


In [21]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊




In [22]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T17:03:10.558887473Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}},{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-19T16:55:09.890394576Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}}]}

In [23]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [24]:
llm = OllamaLLM(model="gemma3:270m")

In [25]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊



In [26]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [27]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [28]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [29]:
chain = prompt | llm

In [30]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [31]:
response

'แน่นอนครับ! นี่คือโค้ด Python ที่สามารถหาผลรวมของตัวเลข 1 ถึง 10 ได้:\n\n```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  total = 0\n  for number in numbers:\n    total += number\n  return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum(numbers)\nprint(result)\n```\n\nโค้ดนี้มีตัวอย่างการใช้งานง่ายๆ เพื่อให้เข้าใจว่าโค้ดทำงานอย่างไร:\n\n1.  **`find_sum(numbers)`**:  นี่คือชื่อของฟังก์ชันที่ใช้ในการหาผลรวมของตัวเลข\n2.  **`total = 0`**:  เรากำหนดค่าเริ่มต้นค่าที่เก็บผลรวม (0)\n3.  **`for number in numbers:`**:  เราจะวนซ้ำผ่านทุกตัวเลขใน `numbers`\n4.  **`total += number`**:  เราจะเพิ่มค่า `number` เข้าไปในค่าคงที่ `total`\n5.  **`return total`**:  เราจะคืนค่าค่าสุดท้ายของ `total` ซึ่งเป็นผลรวมของตัวเลขที่ได้\n6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  เราสร้างชุดของตัวเลขที่ต้องการหาผลรวม\n7.  **`result = find_sum(numbers)`**:  เรียกฟังก์ชัน `find_sum` เพื่อทำการหาผลรวม\n8.  **`print(result)`**:  แสดงผ

In [32]:
from IPython.display import display, Markdown

In [33]:
display(Markdown(response))

แน่นอนครับ! นี่คือโค้ด Python ที่สามารถหาผลรวมของตัวเลข 1 ถึง 10 ได้:

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  total = 0
  for number in numbers:
    total += number
  return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

โค้ดนี้มีตัวอย่างการใช้งานง่ายๆ เพื่อให้เข้าใจว่าโค้ดทำงานอย่างไร:

1.  **`find_sum(numbers)`**:  นี่คือชื่อของฟังก์ชันที่ใช้ในการหาผลรวมของตัวเลข
2.  **`total = 0`**:  เรากำหนดค่าเริ่มต้นค่าที่เก็บผลรวม (0)
3.  **`for number in numbers:`**:  เราจะวนซ้ำผ่านทุกตัวเลขใน `numbers`
4.  **`total += number`**:  เราจะเพิ่มค่า `number` เข้าไปในค่าคงที่ `total`
5.  **`return total`**:  เราจะคืนค่าค่าสุดท้ายของ `total` ซึ่งเป็นผลรวมของตัวเลขที่ได้
6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  เราสร้างชุดของตัวเลขที่ต้องการหาผลรวม
7.  **`result = find_sum(numbers)`**:  เรียกฟังก์ชัน `find_sum` เพื่อทำการหาผลรวม
8.  **`print(result)`**:  แสดงผลลัพธ์ของฟังก์ชันที่คำนวณผลรวม
**สรุป:**

โค้ดนี้ใช้ `for` loop เพื่อวนซ้ำผ่านทุกตัวเลขใน `numbers` และ `find_sum` เพื่อหาผลรวมของตัวเลขเหล่านั้น  ค่า `total` จะถูกเก็บไว้ใน `result` ซึ่งจะถูกนำไปใช้ในการคำนวณผลรวม



In [34]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [35]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [36]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [37]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [38]:
parser_chain = StrOutputParser()

In [39]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [40]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [41]:
result

'การทำจากผลลัทธ์นั้นมีความสำคัญอย่างยิ่งต่อการพัฒนาทางเศรษฐกิจและสังคมในยุคปัจจุบัน การวิจัยและพัฒนาผลลัทธ์ไม่ใช่แค่การหาทางแก้ปัญหา แต่เป็นการสร้างสรรค์สิ่งใหม่ๆ ที่สามารถนำไปสู่ความยั่งยุติธรรม และความเท่าเทียมกันได้'

In [42]:
keyword_prompt = PromptTemplate(
    template = """
    จาก Review ต่อไปนี้
    Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

    Reviews: {review}

    ขั้นตอนการ Extrack keywords
    1. อ่านรีวิวและเข้าใจความหมาย
    2. ระบุหัวข้อหลักที่กล่าวถึง
    3. เลือก 3 keywords ที่สำคัญที่สุด
    4. ตอบในรูปแบบ JSON โดยมี key คือ keywords: {{"keywords": "คำ1, คำ2, คำ3"}}
ให้ทำทีละขั้นตอน
""")

In [43]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [44]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [45]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [46]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    15 seconds ago            


In [47]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [48]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [49]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 62.5 ms, sys: 3.12 ms, total: 65.6 ms
Wall time: 6.46 s


In [50]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "แพงเวอร์",
    "ราคาสูง",
    "ไม่สั่ง",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "ขนมหวาน",
    "ฟรี",
    "ขนมไทย",
    "ราคา150บาท"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text จาก Review นี้, นี่คือ 3 คำแนะนำที่เหมาะสมสำหรับคนไปทานกาแฟร้านวาวี:

1.  "วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆ อารีย์" - คำนี้สื่อถึงความรู้สึกที่ได้รับจากการไปทานกาแฟที่ร้านวาวี ซึ่งเป็นที่ที่คนส่วนใหญ่มารับดูและชื่นชม

2.  "ได้ยินมาบ่อยๆ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก" - คำนี้แสดงให้เห็นว่าการได้ยินมาบ่อยๆ ก็เป็นสิ่งที่ทำให้รู้สึกประทับใจและอยากลองทานกาแฟ

3.  "เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป" - คำนี้เป็นการเปิดเผ

In [51]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [52]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [53]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [54]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [55]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 132 ms, sys: 12.9 ms, total: 145 ms
Wall time: 15 s


In [56]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ขนาด']

In [57]:
results[1]['text']['keywords']

['กาแฟ', 'วาวี', 'อารีย์']

In [58]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [59]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 144 ms, sys: 11 ms, total: 155 ms
Wall time: 13 s


In [60]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาดร้าน'], ['กาแฟ', 'บรรยากาศ', 'วาวี'])

In [61]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [62]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [63]:
inputs = [{"review": r} for r in sample_reviews]

In [64]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 21 ms, sys: 1.21 ms, total: 22.2 ms
Wall time: 2.35 s


In [65]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

# 1000 review

In [66]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [67]:
from tqdm import tqdm

In [68]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:40<00:00,  9.44s/it]


In [69]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [70]:
results_1000[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'}}

In [71]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)


In [72]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [73]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [74]:
context_extract = context_prompt | llm3 | context_parser

In [75]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [76]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [77]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 52937.66it/s]


In [78]:
len(context_inputs)

24827

In [79]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:25:34<00:00,  5.13s/it]


In [80]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [81]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 512250.12it/s]


In [82]:
len(sentiment_inputs)

3000

In [83]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [84]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [42:06<00:00,  2.53s/it]


In [85]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'negative', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรย

In [86]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [87]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')